In [ ]:
#import packages
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import re
import pickle
import sys
from imdb import IMDb
from urllib.parse import urlencode
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [ ]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  omdb_title text,"
    "  bom_title text,"
    "  budget int(10),"
    "  box_office int(15),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nominations int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL AUTO_INCREMENT,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL AUTO_INCREMENT,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

In [ ]:
# get a list of links to movies on box office mojo
# takes in a list of years you want movies from
def retrieve_movie_links(years):
    movie_links = [] #all movie links so far
    links = [] #links from the last page 
    for year in years:
        print("in for loop, year = ", year)
        page = 1
        # enter the loop if it's the first page we're seeing
        # OR enter the loop if links is not empty
        while links or page == 1: 
            print("in while loop, page = ", page)
            #make links list empty
            links = []
            #get the links from current page and add to links list
            html_page = requests.get('https://www.boxofficemojo.com/yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page, year))
            # Pass the page contents to beautiful soup for pars
            soup = BeautifulSoup(html_page.content, 'html.parser')
            for x in list(soup.find_all('tr')):
                for y in x.findChildren('td'):
                    for z in y.findChildren('a'):
                        if 'movies' in str(z.get('href')):
                            links.append(z.get('href'))
 
            # add set of links to the movie_links list (.extend)
            # set because otherwise there will be duplicates
            movie_links.extend(set(links))
            #increase page count
            page += 1
            print("page is ", page, "links:", not links)
            time.sleep(1)
    return movie_links


In [ ]:
# function to replace multiple different substrings with one 
# particular substring in a given string
def replace_multiple(string, to_replace_list, replacement):
    for to_replace in to_replace_list:
        string = string.replace(to_replace, replacement)
    return string

In [ ]:
# scrapes each page on box office mojo for the title and budget,
# given a list of links
def scrape_each_movie_page(movie_links):
    movies = []
    for movie in movie_links:
        
        # show progress
        if movie_links.index(movie) % 10 == 0:
            print("progress:", movie_links.index(movie),"/", len(movie_links))
        # gets movie page
        html_page = requests.get('https://www.boxofficemojo.com{}'.format(movie)) 
        
        # Pass the page contents to beautiful soup for parsing
        soup = BeautifulSoup(html_page.content, 'html.parser')
        
        # find all the elements with the tag 'b', which is where all the good info is
        prelim_findings = list(soup.find_all('tr'))
        
        findings = list(soup.find_all('b'))
        
        # holds our info (title & budget) for the movie we're currently looking at
        movie_dict = {}

In [ ]:
# function for making titles with years in them searchable on OMDB
def find_year_in_title(title):
    # regular expression to find all the years in the OMDB format
    year = re.findall('(201\d)', title)
    # if there's a year, returns a tuple with the year and the 
    # title without the year
    if year:
        return (year[0], title.replace("("+year[0]+")", ""))
    # otherwise it just gives you back the title
    else:
        return title

In [ ]:
# function that returns a list of movie titles from a dictionary of titles and bugets
def get_movie_titles(list_of_movie_dictionaries):
    movie_titles = []
    for movie in list_of_movie_dictionary:
        title_to_search = movie['title']
        # get rid of 'amp' in titles (cleaning up title names for API call)
        if '&amp;' in title_to_search:
            title_to_search = movie.replace('&amp;', '&')
        movie_titles.append(find_year_in_title(title_to_search))
        
    return movie_titles

In [ ]:
# Function to make call to OMDB API using title from BOM

key = config.apiKey
url = 'http://www.omdbapi.com/?i=tt3896198&apikey='

def OMDB_call(titles):
    all_titles = []
    # Iterate over list of titles (from box office mojo) to retrieve all
    for title in titles:
        if type(title) == tuple:
            parameters = {'t': title[0], 'y': title[1]}
        else:
            parameters = {'t': title}
        response = requests.get(url + key, params=parameters)    
        data = response.json()
        data['bom_title'] = title
        all_titles.append(data)
        time.sleep(.5)
        # show progress
        if titles.index(title) % 50 == 0:
            print("progress:", titles.index(title),"/", len(titles))
            filename = 'all_data'
            outfile = open(filename, 'wb')
            pickle.dump(all_titles,outfile)
            outfile.close()
    return all_titles

In [ ]:
# function that checks if list from BOM has actually returned anything from OMDB
def check_for_title(data):
    movies_dict = []
    for movie in movie_dict:
        if 'Title' in movie.keys():
            movies_dict.append(movie)
    return movies_dict

In [ ]:
# function that converts a string into a date
def string_to_time(string):
    return datetime.strftime(datetime.strptime(string, "%d %b %Y"), "%Y-%m-%d")

# function to convert box office revenue from string to integer
def str_to_int(movies_dict):
    try:
        for movie in movies_dict:
            if type(movie['BoxOffice']) != int:
                if movie['BoxOffice'] != 'N/A':
                    movie['BoxOffice'] = int(movie['BoxOffice'][1:].replace(",", ""))
    except KeyError:
        print(movie)
        
# function that cleans up our data from OMDB
def clean_up_data(movies_dict):
    # iterates over a dictionary and converts a particular string to a date
    #
    for movie in movies_dict:
        str_to_int(movies_dict)
        if movie['Released'] != 'N/A':
            movie['Released'] = string_to_time(movie['Released'])
        if 'BoxOffice' not in movie.keys():
            movie['BoxOffice'] = 0
        if 'Rotten Tomatoes' not in [x['Source'] for x in movie['Ratings']]:
            movie['Ratings'].append({'Source': 'Rotten Tomatoes', 'Value':'0%'})
    return movies_dict
                  

In [ ]:
# insert movie data into tables

def add_movies_toDB(movies_dict):
    for movie in movies_dict:
        if type(movie['bom_title']) == tuple:
            movie['bom_title'] = movie['bom_title'][0]
        RT_rating = [x['Value'] for x in movie['Ratings'] if x['Source'] == 'Rotten Tomatoes'][0] 
        cursor.execute('''USE Mod_1_Project''')
        cursor.execute('''insert into main_movie_table (omdb_title, bom_title, box_office, release_date, RT_rating, IMDB_rating) values(%s,%s,%s,%s,%s,%s)''',
        (movie['Title'], movie['bom_title'], movie['BoxOffice'], movie['Released'], RT_rating.replace('%', ''), movie['imdbRating'])) 
    cnx.commit()
    

In [ ]:
# give function a string of money like "$.5 million" and will return 500000
def money_string_to_int(money_string):
    # no commas pls (delete all commas)
    money_string = money_string.replace(',','')
    
    # gets 'million' or 'billion' from string
    money_multiplier = money_string.split(' ')[-1]
    
    # gets the integer from the string
    money_amount = list(map(int, re.findall('\d+', money_string)))
    
    #if the string entered isn't an amount of money (ie "abc" or "n/a")
    if not money_amount:
        #just give it back to them
        return money_string
    elif money_multiplier == 'million':
        return money_amount[0]*1000000
    elif money_multiplier == 'billion':
        return money_amount[0]*100000000
    else:
        # if it is not million or billion, then it's in the thousands, 
        # or less, which is given already in the proper format
        return money_amount[0]

In [ ]:
# update the budget column in the rows that're in the db right now
def update_budget(db_name, budget_list_with_titles):
    for movie in budget_list_with_titles:
        # reformat budget to be an integer
        budget = money_string_to_int(movie['budget'])
        
        # now we can update the budget in the table
        if budget != "N/A" and budget != 'Unknown':
            cursor.execute("USE {}".format(db_name))
            cursor.execute("""UPDATE main_movie_table 
                               SET budget = %s
                               WHERE bom_title = %s
                               """,(budget, movie['title']))
    cnx.commit()
    return

In [ ]:
# set of functions that uploads genre information into database

# find the movie_id for each movie and return int
def movie_id(title):
    title = title.replace(r"'",r"\'")
    cursor.execute('''USE Mod_1_Project''')
    cursor.execute(f"""SELECT movie_id from main_movie_table where omdb_title = '{title}';""")
    return cursor.fetchone()[0]

# get a list of genres associated with a movie
def find_genres(title, movies_dict):
    for movie in movies_dict:
        if title == movie['Title']:
            return movie['Genre']

# add genres to the database if they don't already exist
def add_genres_to_DB(genres):
    cursor.execute('''USE Mod_1_Project''')
    cursor.execute(f"""SELECT genre_id from genres where genre = '{genre}';""")
    genre_id = cursor.fetchone()
    if genre_id is None:
        cursor.execute(f"""INSERT INTO genres (genre) values ('{genre}');""")
        cursor.execute('''SELECT LAST_INSERT_ID()''')
        return cursor.fetchone()[0]
    else:
        return genre_id[0]

# populate join table for movies and genres
def add_movie_genre_toDB(movieid, genreid):
    cursor.execute('''USE Mod_1_Project''')
    cursor.execute(f"""INSERT INTO movies_genres (movie_id, genre_id) values ({movieid}, {genreid});""")
    

In [ ]:
def upload_genres_toDB(movies_dict):
    for movie in movies_dict:
        movieid = movie_id(movie['Title'])
        genres = find_genres(movie['Title'], movies_dict).split(", ")
        for genre in genres:
            genreid = add_genres_to_DB(genre)
            add_movie_genre_toDB(movieid, genreid)

In [ ]:
def get_movie_id(title):
    """SELECT movie_id FROM database matching the title input"""
    title = title.replace(r"'",r"\'")
    cursor.execute(f"""SELECT movie_id FROM main_movie_table WHERE bom_title = '{title}' OR omdb_title = '{title}';""")
    return cursor.fetchone()

def get_actor_id(actor_name):
    """SELECT actor_id FROM database matching actor name"""
    cursor.execute(f"""SELECT actor_id FROM actors WHERE actor_name = '{actor_name}';""")
    return cursor.fetchone()

def list_of_dictionaries_to_df(dictionaries):
    """Read movie data into pandas"""
    return pd.DataFrame(dictionaries)

def add_actors_toDB(omdb_data):
    """Take omdb list of dictionaries and insert actors into database"""
    idx = 0
    sys.stdout.write("INSERTING ACTORS")
    sys.stdout.flush()
    #itertuples returns an iterator of named tuples accessed by attribute
    for movie in omdb_data.itertuples():
        try:
            #We found the inclusion of actors in documentaries to be extraneous
            if movie.Genre != 'Documentary':
                for actor in movie.Actors.split(', '):
                    actor = actor.replace(r"'",r"\'")
                    #get actor_id if already present in database
                    actor_id = get_actor_id(actor)
                    #if actor not present, insert and get id back
                    if actor_id is None:
                        actor_insert_q = f"""INSERT INTO actors(actor_name) VALUES('{actor}');"""
                        cursor.execute(actor_insert_q)
                        cursor.execute("""SELECT LAST_INSERT_ID();""")
                        actor_id = cursor.fetchone()[0]
                    else:
                    # if actor_id is present, strip sql result from tuple
                        actor_id = actor_id[0]

                    #get id of movie
                    movie_id = get_movie_id(movie.Title)[0]

                    insert_actor_movie_q = f"""INSERT INTO movie_actors(movie_id, actor_id) VALUES ('{movie_id}', '{actor_id}');"""
                    #add actor + movie to movie_actors
                    cursor.execute(insert_actor_movie_q)
                    idx += 1
                    if idx % 112 == 0:
                        sys.stdout.write(".")
                        sys.stdout.flush()
        except TypeError:
            print('Non-iterable actors movie', movie)
    #add_actors_toDB()  
    cnx.commit()
    sys.stdout.write("OK")
    sys.stdout.flush()


In [ ]:
# Get titles from database and update with awards info 
def get_awards(movie_title, ia):
    """Query imdb API for award info based on movie title"""
    #If movie doesn't match, return None
    try:
        movie = ia.search_movie(movie_title)[0]
    except IndexError:
        return None
    url_stem = 'https://www.imdb.com/title/tt'
    #soupify request
    soup = BeautifulSoup(requests.get(url_stem+movie.getID()+'/awards').text)
    #awards text is nested in the div with id of main, in a div with a class of desc
    try:
        awards_text = soup.select('#main .desc')[0].string
    except IndexError:
        return None
    #awards sentence is always the same
    awards_words = awards_text.split()
    #the fourth word is either 'win' or 'wins', the third word is the number of wins
    #the seventh word is either 'nomination' or 'nominations', the sixth word is the number of noms
    try:
        return {awards_words[3]:int(awards_words[2]), awards_words[6]:int(awards_words[5])}
    except IndexError:
        return None

def update_movies_with_awards():
    """SELECT titles FROM database; QUERY imdb for awards info; UPDATE database recors"""
    #handle beginning of progress bar
    idx = 0
    sys.stdout.write('[')
    sys.stdout.flush()
    #initiate imdb instance
    ia = IMDb()
    #get titles
    cursor.execute("""SELECT bom_title FROM main_movie_table WHERE award_nominations IS NULL;""")
    #remove titles from tuples    
    titles = [title[0] for title in cursor.fetchall()]
    #wrap code in a try-finally so if error, commit no matter what
    try:
        for title in titles:
            #Error when get_awards returns None. When this happens, 
            #write NULL to db win and nom variables are set using 
            #keys from the awards dictionary. Its a little hacky, 
            #but they will always match
            try:
                win = list(get_awards(title,ia).keys())[0]
                nom = list(get_awards(title,ia).keys())[1]
                award_wins = get_awards(title,ia)[win]
                award_noms = get_awards(title,ia)[nom]
            except Exception as e:
                award_wins = 'NULL'
                award_noms = 'NULL'
            #update awards in database based on title
            title = title.replace(r"'", r"\'")
            award_update_q = f"""UPDATE main_movie_table 
                                SET award_wins = {award_wins}, award_nominations = {award_noms}
                                WHERE bom_title = '{title}';"""
            cursor.execute(award_update_q)
            #more progress bar handling
            idx += 1
            if idx % 23 == 0:
                #while paused for progress bar, also open and close
                #db connection, it had been failing and opening and
                #closing seemed to help
                cnx.commit()
                cursor.close()
                cnx.close()
                
                cnx = mysql.connector.connect(
                    host = config.host,
                    user = config.user,
                    passwd = config.password,
                    database = 'Mod_1_Project',
                    buffered = True
                )
                cursor = cnx.cursor()
                #more progress bar handling
                sys.stdout.write('=')
                sys.stdout.flush()
    finally:
        cnx.commit()
        sys.stdout.write(']')
        sys.stdout.flush()

In [ ]:
# database set up
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password)

# Establish cursor and name database
cursor = cnx.cursor(buffered=True)
db_name = 'Mod_1_Project'



# web scraping
years = [2017, 2018, 2019]
links = retrieve_movie_links(years)
mojo_info = scrape_each_movie_page(links)
movie_titles = get_movie_titles(mojo_info)
omdb_info = check_for_title(OMDB_call(movie_titles))
cleaned_omdb_info = clean_up_data(omdb_info)



# create and populate database
create_database(cursor, db_name)
add_movies_toDB(cleaned_omdb_info)
update_budget(db_name, mojo_info)
upload_genres_toDB(cleaned_omdb_info)
omdb_dataframe = list_of_dictionaries_to_df(cleaned_omdb_data)
add_actors_toDB(omdb_dataframe)
update_movies_with_awards()